# Lab 3 — dimensionality reduction

In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np

from operator import itemgetter

%matplotlib inline
plt.style.use("ggplot")

In [2]:
!hdfs dfs -cat /ix/ml-20m/genome-tags.txt | tail -n 2

{"tagId": 1127, "tag": "zombie"}
{"tagId": 1128, "tag": "zombies"}


movies -> movieId, title, [genre]
tags -> tagId, tag
scores -> (movieId, tagId), relevance

In [3]:
movies = sc.textFile("/ix/ml-20m/movies.txt").map(json.loads)
tags = sc.textFile("/ix/ml-20m/genome-tags.txt").map(json.loads)
scores = sc.textFile("/ix/ml-20m/genome-scores.txt").map(json.loads)
ratings = sc.textFile("/ix/ml-20m/ratings.txt").map(json.loads)

In [64]:
movie1 = movies.filter(lambda x: x['movieId']==1)

In [66]:
movie1.collect()

[{'genres': ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'],
  'movieId': 1,
  'title': 'Toy Story (1995)'}]

In [68]:
tags.collect()

[{'tag': '007', 'tagId': 1},
 {'tag': '007 (series)', 'tagId': 2},
 {'tag': '18th century', 'tagId': 3},
 {'tag': '1920s', 'tagId': 4},
 {'tag': '1930s', 'tagId': 5},
 {'tag': '1950s', 'tagId': 6},
 {'tag': '1960s', 'tagId': 7},
 {'tag': '1970s', 'tagId': 8},
 {'tag': '1980s', 'tagId': 9},
 {'tag': '19th century', 'tagId': 10},
 {'tag': '3d', 'tagId': 11},
 {'tag': '70mm', 'tagId': 12},
 {'tag': '80s', 'tagId': 13},
 {'tag': '9/11', 'tagId': 14},
 {'tag': 'aardman', 'tagId': 15},
 {'tag': 'aardman studios', 'tagId': 16},
 {'tag': 'abortion', 'tagId': 17},
 {'tag': 'absurd', 'tagId': 18},
 {'tag': 'action', 'tagId': 19},
 {'tag': 'action packed', 'tagId': 20},
 {'tag': 'adaptation', 'tagId': 21},
 {'tag': 'adapted from:book', 'tagId': 22},
 {'tag': 'adapted from:comic', 'tagId': 23},
 {'tag': 'adapted from:game', 'tagId': 24},
 {'tag': 'addiction', 'tagId': 25},
 {'tag': 'adolescence', 'tagId': 26},
 {'tag': 'adoption', 'tagId': 27},
 {'tag': 'adultery', 'tagId': 28},
 {'tag': 'adventur

In [4]:
print('Number of movies:', movies.count())
print('Number of tags:', tags.count())
print('Number of scores:', scores.count())
print('Number of ratings:', ratings.count())

Number of movies: 27278
Number of tags: 1128
Number of scores: 11709768
Number of ratings: 20000263


Construct MxN matrix, where N=1128 - tagId, M=10381 - movieId with at least one tag

In [30]:
sorted_scores = scores.map(lambda x: (x['movieId'], (x['tagId'], x['relevance']))).groupByKey().sortByKey()

In [31]:
sorted_scores.take(1)

[(1, <pyspark.resultiterable.ResultIterable at 0x7f9b40d95710>)]

M:

In [32]:
sorted_scores.count()

10381

In [15]:
matrix = np.zeros((10381, 1128))

In [18]:
matrix.shape

(10381, 1128)

In [41]:
s = sorted_scores.collect()

In [44]:
for i in range(len(s)):
    for j,r in s[i][1]:
        matrix[i][j-1] = r

In [46]:
list(s[0][1])

[(1, 0.025000000000000022),
 (2, 0.025000000000000022),
 (3, 0.057750000000000024),
 (4, 0.09675),
 (5, 0.14675),
 (6, 0.21700000000000003),
 (7, 0.067),
 (8, 0.26275000000000004),
 (9, 0.262),
 (10, 0.03199999999999997),
 (11, 0.577),
 (12, 0.11625000000000002),
 (13, 0.188),
 (14, 0.008000000000000007),
 (15, 0.036750000000000005),
 (16, 0.28174999999999994),
 (17, 0.007000000000000006),
 (18, 0.11049999999999999),
 (19, 0.6705),
 (20, 0.1845),
 (21, 0.33025000000000004),
 (22, 0.2825),
 (23, 0.056999999999999995),
 (24, 0.015500000000000014),
 (25, 0.08500000000000002),
 (26, 0.08100000000000002),
 (27, 0.195),
 (28, 0.07150000000000001),
 (29, 0.8919999999999999),
 (30, 0.67625),
 (31, 0.03875000000000001),
 (32, 0.22799999999999998),
 (33, 0.402),
 (34, 0.03875000000000001),
 (35, 0.026749999999999996),
 (36, 0.33025000000000004),
 (37, 0.10099999999999998),
 (38, 0.012500000000000011),
 (39, 0.018500000000000016),
 (40, 0.014249999999999985),
 (41, 0.013749999999999984),
 (42, 0.

In [50]:
print(matrix)

[[ 0.025    0.025    0.05775 ...,  0.03625  0.07775  0.023  ]
 [ 0.03975  0.04375  0.03775 ...,  0.01475  0.09025  0.01875]
 [ 0.0435   0.05475  0.028   ...,  0.0195   0.097    0.0185 ]
 ..., 
 [ 0.042    0.03175  0.017   ...,  0.0095   0.0755   0.01625]
 [ 0.05975  0.10375  0.1785  ...,  0.51925  0.067    0.01725]
 [ 0.05325  0.08625  0.017   ...,  0.01575  0.1145   0.02175]]


In [18]:
print(movies.filter(lambda x: len(x['genres']) > 0).count())
print(scores.groupByKey().take(3))

27278


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 25.0 failed 4 times, most recent failure: Lost task 0.3 in stage 25.0 (TID 120, iccluster063.iccluster.epfl.ch, executor 3): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/hdp/current/spark2-client/python/pyspark/worker.py", line 253, in main
    process()
  File "/usr/hdp/current/spark2-client/python/pyspark/worker.py", line 248, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/hdp/current/spark2-client/python/pyspark/rdd.py", line 2440, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/hdp/current/spark2-client/python/pyspark/rdd.py", line 2440, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/hdp/current/spark2-client/python/pyspark/rdd.py", line 350, in func


In [10]:
b = False
def add_to_mat(m, x):
    if not b:
        print('starting')
        b = True
    m[x['movieId'], x['tagId']] = x['relevance']

In [14]:
scores_list = scores.groupBy(lambda x: x['movieId']).collect()

KeyboardInterrupt: 